In [1]:
import numpy as np
import matplotlib.pylab as plt
import matplotlib as mpl

def set_plot_prop():
    plt.ioff()
    mm2inch = lambda x: x/10./2.54
    # plt.rcParams['xtick.direction']= 'out'
    # plt.rcParams['ytick.direction']= 'out'
    plt.rcParams['axes.grid'] = True
    plt.rcParams['xtick.labelsize'] = 20
    plt.rcParams['ytick.labelsize'] = 20
    plt.rcParams['axes.labelsize'] = 24
    plt.rcParams['grid.color'] = 'k'
    plt.rcParams['grid.linestyle'] = ':'
    plt.rcParams['grid.linewidth'] = 0.75
    # plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['font.size'] = 24
    plt.rcParams['lines.linewidth'] = 1.5
    plt.rcParams['axes.linewidth'] = 2.
    plt.rcParams['figure.figsize'] = mm2inch(90*5),mm2inch(2./3*90*5)
    plt.rcParams["legend.handlelength"] = 1.
    plt.rcParams["legend.handletextpad"] = 0.15
    plt.rcParams["legend.borderpad"] = 0.15
    plt.rcParams["legend.labelspacing"] = 0.15
    cmap= mpl.colormaps.get_cmap('RdYlBu')
    plt.rcParams.update({
        "figure.facecolor":  (1.0, 1.0, 1.0, 1),  # red   with alpha = 30%
        "axes.facecolor":    (1.0, 1.0, 1.0, 1),  # green with alpha = 50%
        "savefig.facecolor": (1.0, 1.0, 1.0, 1),  # blue  with alpha = 20%
    })
    plt.rcParams['axes.facecolor'] = 'white'
    # Set the default color cycle
    mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["navy", "blue", 'lightseagreen',"sandybrown",'salmon','red','maroon']) 


set_plot_prop()

# Sensitivity of DIC to pH and *p*CO<sub>2</sub>

*Based on **CO2SYSExample1.m** for MATLAB by Steven van Heuven.*

## Introduction

This is an example of the use of PyCO2SYS that uses its ability to process arrays of data.

We will generate a figure that shows the sensitivity of pH and *p*CO<sub>2</sub> to changes in DIC, while keeping everything else constant.

You can find further information about this way of using PyCO2SYS [in its documentation](https://pyco2sys.readthedocs.io/en/latest/co2sys_nd/).

## Define input conditions

The first step is to define the input conditions that we want to use with PyCO2SYS. In this case, every input has a single constant value except for DIC (`par2`), which is a NumPy array of values increasing from 2100 to 2300 μmol·kg<sup>-1</sup> in increments of 5 μmol·kg<sup>-1</sup>:

In [139]:
## Import NumPy to make the DIC array
import numpy as np
# Import PyCO2SYS
import PyCO2SYS as pyco2

# Import plotting package
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline

def make_range(pressure,temperature,pressure_name,co_scale,ph_scale,step_dic=5,step_ta=5,plot_pdf=False):
    factor = 1e-3
    #arrow start and end points
    x_start = np.array([2025, 2025, 2025, 2025, 2025])*factor-25*factor
    y_start = np.array([2000, 2000, 2000,2000,2000])*factor+25*factor
    x_end = np.array([2050, 2000,2025, 2000, 2050])*factor-25*factor
    y_end = np.array([2050, 1950,1950, 2000, 2000])*factor+25*factor
    
    #creating the arrows
    texts = ['CaCO$_3$ \n dissolution','CaCO$_3$ \n precipitation', 'Sulfate addition',r'CO$_2$ release',r'CO$_2$ invasion']
    x_texts = np.array([2000, 1925,2040, 1905, 2050])*factor+25*factor
    y_texts = np.array([2055, 1935,2050-100, 1995, 1995])*factor+25*factor
    
    
    #texts2 = [r'+2','-2', '-2',r'',r'']
    #x_texts2 = np.array([2020, 2000,2030, 1945, 2050])*factor-75*factor 
    #y_texts2 = np.array([2025, 1978,1935, 1995, 1995])*factor+100*factor

    
    plt.close('all')
    fig1 = plt.figure(1,figsize=(10,12))
    ax1 = plt.gca()
    fig2 = plt.figure(2,figsize=(10,12))
    ax2 = plt.gca()
    divider1 = make_axes_locatable(ax1)
    divider2 = make_axes_locatable(ax2)
    cax1 = divider1.append_axes('right', size='5%', pad=0.05)
    cax2 = divider2.append_axes('right', size='5%', pad=0.05)

    # try :
    #     del results, kwargs
    # except :
    #     continue
    dic_vals = np.arange(1800, 2205, step_dic)
    for dic in dic_vals:
        # Define input conditions
        kwargs = dict(
            par1 = np.arange(1850, 2205, step_ta),  # Value of the first parameter
            par2 = dic,  # Value of the second parameter, which is a long vector of different DIC's!
            par1_type = 1,  # The first parameter supplied is of type "1", which is "alkalinity"
            par2_type = 2,  # The second parameter supplied is of type "2", which is "DIC"
            salinity = 35,  # Salinity of the sample
            pressure = pressure, ## Pressure in dbar for the sample
            temperature = temperature,  # Temperature at input conditions
            total_silicate = 50,  # Concentration of silicate  in the sample (in umol/kg)
            total_phosphate = 2,  # Concentration of phosphate in the sample (in umol/kg)
            opt_k_carbonic = 4,  # Choice of H2CO3 and HCO3- dissociation constants K1 and K2 ("4" means "Mehrbach refit") 
            # Doesn't affect very much based on testing a few options
            opt_k_bisulfate = 1,  # Choice of HSO4- dissociation constants KSO4 ("1" means "Dickson")
        )

        #print("Input conditions have been set!")
        # Run CO2SYS!
        results = pyco2.sys(**kwargs)
        #print('PyCO2SYS ran successfully!')
        # The calculated pCO2's are in the field 'pCO2' of the results CO2dict
        # Show these in the first subplot
        results['par1'] = results['par1']*factor
        results['par2'] = results['par2']*factor
        f1 = ax1.scatter('par2','par1', data=results, c='pCO2',cmap='RdYlBu_r',s=90,vmin=co2_scale[0],vmax=co2_scale[1])
        if dic == dic_vals[-1] :
            plt.colorbar(f1, cax=cax1,label=r'pCO$_2$ [ppm]')
            ax1.set_xlabel(r"Dissolved Inorganic Carbon [mmol kg$^{-1}$]")
            ax1.set_ylabel(r"Total Alkalinity [mmol kg$^{-1}$]")
            ax1.set_title(f'{pressure_name}, {temperature}'+r'$^{o}$ C')
            ax1.set_xlim([1800*factor,2200*factor])
            ax1.set_ylim([1850*factor,2200*factor])
            for i in range(len(x_start)):
              ax1.annotate("", xy=(x_end[i], y_end[i]), xytext=(x_start[i], y_start[i]), textcoords='data',xycoords='data',arrowprops=dict(arrowstyle="->",linewidth=4))
              #ax1.text(x_texts2[i], y_texts2[i],texts2[i], fontsize=20)
              ax1.text(x_texts[i], y_texts[i],texts[i], fontsize=20,horizontalalignment='center')
            fig1.savefig(f'0_results/pCO2_DIC_Alkalinity_{pressure}db_{temperature}C.png',bbox_inches='tight')
            if plot_pdf :
                fig1.savefig(f'0_results/pCO2_DIC_Alkalinity_{pressure}db_{temperature}C.pdf',bbox_inches='tight')

        # # The calculated pH's are in the field 'pH' of the results CO2dict
        f2 = ax2.scatter('par2','par1', data=results, c='pH',cmap='RdYlBu_r',s=90,vmin=ph_scale[0],vmax=ph_scale[1])
        if dic == dic_vals[-1] :
            plt.colorbar(f2, cax=cax2,label=r'pH')
            ax2.set_xlabel(r"Dissolved Inorganic Carbon [mmol kg$^{-1}$]")
            ax2.set_ylabel(r"Total Alkalinity [mmol kg$^{-1}$]")
            ax2.set_title(f'{pressure_name}, {temperature}'+r'$^{o}$ C')
            ax2.set_xlim([1800*factor,2200*factor])
            ax2.set_ylim([1850*factor,2200*factor])
            
            for i in range(len(x_start)):
              ax2.annotate("", xy=(x_end[i], y_end[i]), xytext=(x_start[i], y_start[i]), textcoords='data',xycoords='data',arrowprops=dict(arrowstyle="->",linewidth=4))
              #ax2.text(x_texts2[i], y_texts2[i],texts2[i], fontsize=20)
              ax2.text(x_texts[i], y_texts[i],texts[i], fontsize=20,horizontalalignment='center')
            
            fig2.savefig(f'0_results/pH_DIC_Alkalinity_{pressure}db_{temperature}C.png',bbox_inches='tight')
            if plot_pdf :
                fig2.savefig(f'0_results/pH_DIC_Alkalinity_{pressure}db_{temperature}C.pdf',bbox_inches='tight')

    # # The calculated pH's are in the field 'pH' of the results CO2dict
    plt.close('all')
    plt.close('all')

## Run PyCO2SYS

Once we have defined the input conditions above, solving the marine carbonate system is as simple as importing and running the `pyco2.sys` function:

In [157]:
pH_scale=np.array([6.6,8.5])
co2_scale=np.array([100,8000])
plt.close('all')
pressure = 100
temperature = 10
pressure_name = 'Shallow Ocean'
make_range(pressure,temperature,pressure_name,co2_scale,pH_scale,step_dic=5,step_ta=5)

pressure = 1000
temperature = 10
pressure_name = 'Intermediate Ocean'
make_range(pressure,temperature,pressure_name,co2_scale,pH_scale,step_dic=5,step_ta=5)

pressure = 4000
temperature = 10
pressure_name = 'Deep Ocean'
make_range(pressure,temperature,pressure_name,co2_scale,pH_scale,step_dic=5,step_ta=5)


In [158]:
pH_scale=np.array([6.6,8.5])
co2_scale=np.array([100,8000])

pressure = 100
temperature = 5
pressure_name = 'Shallow Ocean'
make_range(pressure,temperature,pressure_name,co2_scale,pH_scale,step_dic=5,step_ta=5)

pressure = 1000
temperature = 5
pressure_name = 'Intermediate Ocean'
make_range(pressure,temperature,pressure_name,co2_scale,pH_scale,step_dic=5,step_ta=5)

pressure = 4000
temperature = 5
pressure_name = 'Deep Ocean'
make_range(pressure,temperature,pressure_name,co2_scale,pH_scale,step_dic=5,step_ta=5)


In [149]:
pH_scale=np.array([6.6,8.5])
co2_scale=np.array([100,8000])

pressure = 100
temperature = 20
pressure_name = 'Shallow Ocean'
make_range(pressure,temperature,pressure_name,co2_scale,pH_scale,step_dic=5,step_ta=5)

pressure = 1000
temperature = 20
pressure_name = 'Intermediate Ocean'
make_range(pressure,temperature,pressure_name,co2_scale,pH_scale,step_dic=5,step_ta=5)

pressure = 4000
temperature = 20
pressure_name = 'Deep Ocean'
make_range(pressure,temperature,pressure_name,co2_scale,pH_scale,step_dic=5,step_ta=5)


In [150]:
pH_scale=np.array([6.6,8.5])
co2_scale=np.array([100,8000])

pressure = 100
temperature = 25
pressure_name = 'Shallow Ocean'
make_range(pressure,temperature,pressure_name,co2_scale,pH_scale,step_dic=5,step_ta=5)

pressure = 1000
temperature = 25
pressure_name = 'Intermediate Ocean'
make_range(pressure,temperature,pressure_name,co2_scale,pH_scale,step_dic=5,step_ta=5)

pressure = 4000
temperature = 25
pressure_name = 'Deep Ocean'
make_range(pressure,temperature,pressure_name,co2_scale,pH_scale,step_dic=5,step_ta=5)


### Line Plots

In [155]:
# Import NumPy to make the DIC array
import numpy as np
# Import PyCO2SYS
import PyCO2SYS as pyco2

# Import plotting package
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline


def make_range_LinePlots(temperatures,step_dic=5,step_ta=5):
    plt.close('all')
    plt.close('all')
    for dic in np.array([1900, 2000, 2110]):
        fig, ax = plt.subplots(2, 2, figsize=(15, 15),sharex=True)
        for pressure,pressure_name in zip(np.array([0,4000]),np.array(['Shallow Ocean (~100 m)','Deep Ocean (~ 4000 m)'])) :
            for temperature in temperatures :
                # Define input conditions
                kwargs = dict(
                    par1 = np.arange(1810, 2205, step_ta),  # Value of the first parameter
                    par2 = dic,  # Value of the second parameter, which is a long vector of different DIC's!
                    par1_type = 1,  # The first parameter supplied is of type "1", which is "alkalinity"
                    par2_type = 2,  # The second parameter supplied is of type "2", which is "DIC"
                    salinity = 35,  # Salinity of the sample
                    temperature = temperature,  # Temperature at input conditions
                    pressure = pressure, ## Pressure in dbar for the sample
                    total_silicate = 50,  # Concentration of silicate  in the sample (in umol/kg)
                    total_phosphate = 2,  # Concentration of phosphate in the sample (in umol/kg)
                    opt_k_carbonic = 4,  # Choice of H2CO3 and HCO3- dissociation constants K1 and K2 ("4" means "Mehrbach refit") # try 5, and 10
                    opt_k_bisulfate = 1,  # Choice of HSO4- dissociation constants KSO4 ("1" means "Dickson")
                )
                results = pyco2.sys(**kwargs)
                results['par1'] = results['par1']*1e-3
                results['par2'] = results['par2']*1e-3
                if pressure == 0:
                    i1 = 0
                else :
                    i1=1
                    ax[i1][0].set_xlabel(r"Total Alkalinity [mmol kg$^{-1}$]")
                    ax[i1][1].set_xlabel(r"Total Alkalinity [mmol kg$^{-1}$]")
                # Prepare an empty figure
                # Show these in the first subplot
                ax[i1][0].plot('par1', 'pCO2', data=results, marker='o',label=f'{temperature}'+r'$^{o}$ C')
                #ax[0].set_xlabel(r"Total Alkalinity [mmol kg$^{-1}$]")
                ax[i1][0].set_title(f"DIC = {int(results['dic'][0]/1e3)}"+ r" mmol kg$^{-1}$," + f'\n {pressure_name}',fontsize=20);
                ax[i1][0].set_ylabel(r"pCO$_{2}$ [ppm]")

                # The calculated pH's are in the field 'pH' of the results CO2dict
                # Show these in the second subplot
                ax[i1][1].plot('par1', 'pH', data=results, marker='o',label=f'{temperature}'+r'$^{o}$ C')
                ax[i1][1].set_ylabel("pH");
                ax[i1][1].set_title(f"DIC = {int(results['dic'][0]/1e3)}"+ r" mmol kg$^{-1}$," + f'\n {pressure_name}',fontsize=20);
                ax[i1][1].set_ylim([6.6,8.6])
                
            #ax[i1][0].legend()
        ax[i1][1].legend()
        plt.subplots_adjust(wspace=0.25, hspace=0.15)
        
        fig.savefig(f'0_results/Line_Plot_pH_DIC_Alkalinity_Multi_dic{dic}.png',bbox_inches='tight')
        #fig.savefig(f'0_results/Line_Plot_pH_DIC_Alkalinity_Multi_dic{dic}.pdf',bbox_inches='tight')
    plt.close('all')

In [156]:
make_range_LinePlots(np.array([5,10,15,20,25,30,35]))

## Line Plot Main text

In [153]:
# Import NumPy to make the DIC array
import numpy as np
# Import PyCO2SYS
import PyCO2SYS as pyco2

# Import plotting package
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline


def make_range_LinePlots(temperatures,step_dic=5,step_ta=5):
    plt.close('all')
    plt.close('all')
    for dic in np.array([1900, 2000, 2110]):
        fig, ax = plt.subplots(1, 2, figsize=(15, 7.5),sharex=True)
        for pressure,pressure_name in zip(np.array([100]),np.array(['Shallow Ocean'])) :
            for temperature in temperatures :
                # Define input conditions
                kwargs = dict(
                    par1 = np.arange(1810, 2205, step_ta),  # Value of the first parameter
                    par2 = dic,  # Value of the second parameter, which is a long vector of different DIC's!
                    par1_type = 1,  # The first parameter supplied is of type "1", which is "alkalinity"
                    par2_type = 2,  # The second parameter supplied is of type "2", which is "DIC"
                    salinity = 35,  # Salinity of the sample
                    temperature = temperature,  # Temperature at input conditions
                    pressure = pressure, ## Pressure in dbar for the sample
                    total_silicate = 50,  # Concentration of silicate  in the sample (in umol/kg)
                    total_phosphate = 2,  # Concentration of phosphate in the sample (in umol/kg)
                    opt_k_carbonic = 4,  # Choice of H2CO3 and HCO3- dissociation constants K1 and K2 ("4" means "Mehrbach refit") # try 5, and 10
                    opt_k_bisulfate = 1,  # Choice of HSO4- dissociation constants KSO4 ("1" means "Dickson")
                )
                results = pyco2.sys(**kwargs)
                results['par1'] = results['par1']*1e-3
                results['par2'] = results['par2']*1e-3
                if pressure == 0:
                    i1 = 0
                else :
                    i1=1
                    ax[0].set_xlabel(r"Total Alkalinity [mmol kg$^{-1}$]")
                    ax[1].set_xlabel(r"Total Alkalinity [mmol kg$^{-1}$]")
                # Prepare an empty figure
                # Show these in the first subplot
                ax[0].plot('par1', 'pCO2', data=results, marker='o',label=f'{temperature}'+r'$^{o}$ C')
                #ax[0].set_xlabel(r"Total Alkalinity [mmol kg$^{-1}$]")
                ax[0].set_title(f"DIC = {int(results['dic'][0]/1e3)}"+ r" mmol kg$^{-1}$," + f'\n {pressure_name}',fontsize=20);
                ax[0].set_ylabel(r"pCO$_2$ [ppm]")

                # The calculated pH's are in the field 'pH' of the results CO2dict
                # Show these in the second subplot
                ax[1].plot('par1', 'pH', data=results, marker='o',label=f'{temperature}'+r'$^{o}$ C')
                ax[1].set_ylabel("pH");
                ax[1].set_title(f"DIC = {int(results['dic'][0]/1e3)}"+ r" mmol kg$^{-1}$," + f'\n {pressure_name}',fontsize=20);
                ax[1].set_ylim([6.6,8.6])
                
            #ax[i1][0].legend()
        ax[1].legend()
        plt.subplots_adjust(wspace=0.25, hspace=0.15)
        
        fig.savefig(f'0_results/2Panel_Line_Plot_pH_DIC_Alkalinity_Multi_dic{dic}.png',bbox_inches='tight')
        fig.savefig(f'0_results/2Panel_Line_Plot_pH_DIC_Alkalinity_Multi_dic{dic}.pdf',bbox_inches='tight')
    plt.close('all')

In [154]:
make_range_LinePlots(np.array([5,10,15,20,25,30,35]))

### Notes

In [31]:
# # Define input conditions
# kwargs = dict(
#     par1 = np.arange(2100, 2305, 5),  # Value of the first parameter
#     par2 = 2150,  # Value of the second parameter, which is a long vector of different DIC's!
#     par1_type = 1,  # The first parameter supplied is of type "1", which is "alkalinity"
#     par2_type = 2,  # The second parameter supplied is of type "2", which is "DIC"
#     salinity = 35,  # Salinity of the sample
#     temperature = 30,  # Temperature at input conditions
#     total_silicate = 50,  # Concentration of silicate  in the sample (in umol/kg)
#     total_phosphate = 2,  # Concentration of phosphate in the sample (in umol/kg)
#     opt_k_carbonic = 4,  # Choice of H2CO3 and HCO3- dissociation constants K1 and K2 ("4" means "Mehrbach refit")
#     opt_k_bisulfate = 1,  # Choice of HSO4- dissociation constants KSO4 ("1" means "Dickson")
# )
# print("Input conditions have been set!")

# kwargs2 = dict(
#     par1 = np.arange(2100, 2305, 5),  # Value of the first parameter
#     par2 = 2150,  # Value of the second parameter, which is a long vector of different DIC's!
#     par1_type = 1,  # The first parameter supplied is of type "1", which is "alkalinity"
#     par2_type = 2,  # The second parameter supplied is of type "2", which is "DIC"
#     salinity = 35,  # Salinity of the sample
#     temperature = 5,  # Temperature at input conditions
#     total_silicate = 50,  # Concentration of silicate  in the sample (in umol/kg)
#     total_phosphate = 2,  # Concentration of phosphate in the sample (in umol/kg)
#     opt_k_carbonic = 4,  # Choice of H2CO3 and HCO3- dissociation constants K1 and K2 ("4" means "Mehrbach refit")
#     opt_k_bisulfate = 1,  # Choice of HSO4- dissociation constants KSO4 ("1" means "Dickson")
# )

# # Import PyCO2SYS
# import PyCO2SYS as pyco2

# # Run CO2SYS!
# results = pyco2.sys(**kwargs)
# results2 = pyco2.sys(**kwargs2)

# print('PyCO2SYS ran successfully!')


# # Import plotting package
# from matplotlib import pyplot as plt
# %matplotlib inline

# # Prepare an empty figure
# fig, ax = plt.subplots(2, 1, figsize=(6, 7))

# # The calculated pCO2's are in the field 'pCO2' of the results CO2dict
# # Show these in the first subplot
# ax[0].plot('par1', 'pCO2', data=results, c='r', marker='o')
# ax[0].plot('par1', 'pCO2', data=results2, c='b', marker='o')
# ax[0].set_xlabel("alka")
# ax[0].set_ylabel("pCO2 [uatm]")

# # The calculated pH's are in the field 'pH' of the results CO2dict
# # Show these in the second subplot
# ax[1].plot('par1', 'pH', data=results, c='r', marker='o')
# ax[1].plot('par1', 'pH', data=results2, c='b', marker='o')
# ax[1].set_xlabel("alka")
# ax[1].set_ylabel("pH");